In [12]:
from datetime import datetime, time, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from utils import *
from median_optimization import optimal_median

In [13]:
TRAIN_START = "2023-10-02 00:00"
TRAIN_END = "2023-12-10 23:59"

TEST_START = "2023-10-28 00:00"
TEST_END = "2023-10-31 23:59"

PUBLIC_START = "2023-10-21 00:00"
PUBLIC_END = "2023-10-24 23:59"

PRIVATE_START = "2023-12-11 00:00"
PRIVARE_END = "2023-12-17 23:59"

In [14]:
with open("./cache/small_data_cache.pkl", "rb") as f:
    df = pd.read_pickle(f)
with open("../html.2023.final.data/sno_test_set.txt") as f:
    ntu_snos = [l.strip() for l in f.read().splitlines()]
with open("./cache/10-03_12_09_rain.pkl","rb") as f:
    rain_df = pd.read_pickle(f)

In [15]:
print(rain_df)
print(df)

                datetime  rain
0    2023-10-03 01:00:00   0.0
1    2023-10-03 02:00:00   0.0
2    2023-10-03 03:00:00   0.0
3    2023-10-03 04:00:00   0.0
4    2023-10-03 05:00:00   0.0
...                  ...   ...
2227 2023-12-09 20:00:00   0.0
2228 2023-12-09 21:00:00   0.0
2229 2023-12-09 22:00:00   0.0
2230 2023-12-09 23:00:00   0.0
2231 2023-12-10 00:00:00   0.0

[1632 rows x 2 columns]
                    time        sno  tot  sbi act
0    2023-10-02 00:00:00  500101001   28   12   1
1    2023-10-02 00:01:00  500101001   28   12   1
2    2023-10-02 00:02:00  500101001   28   13   1
3    2023-10-02 00:03:00  500101001   28   13   1
4    2023-10-02 00:04:00  500101001   28   13   1
...                  ...        ...  ...  ...  ..
1395 2023-12-10 23:15:00  500119091   18    0   1
1396 2023-12-10 23:16:00  500119091   18    0   1
1397 2023-12-10 23:17:00  500119091   18    0   1
1398 2023-12-10 23:18:00  500119091   18    0   1
1399 2023-12-10 23:19:00  500119091   18    0   1

[9

In [89]:
ntu_tots = get_tot(df, ntu_snos)
# the data looks like this:
"""
     datetime               sno      tot   sbi   bemp  act
0    2023-10-02 00:00:00  500101001  28.0  12.0  16.0   1
1    2023-10-02 00:01:00  500101001  28.0  12.0  16.0   1
2    2023-10-02 00:02:00  500101001  28.0  13.0  15.0   1
...
"""
holidays = [d for d in date_range(start=TRAIN_START, end=PRIVARE_END) if is_holiday(d)]

In [16]:
df['datehour'] = df['time'].dt.floor("H")


In [17]:
rain_df.rename(columns={'datetime':'datehour'},inplace=True)


argument rain to df

In [18]:
df = df.merge(rain_df,on='datehour',how='left')
print(df)

                       time        sno  tot  sbi act            datehour  rain
0       2023-10-02 00:00:00  500101001   28   12   1 2023-10-02 00:00:00   NaN
1       2023-10-02 00:01:00  500101001   28   12   1 2023-10-02 00:00:00   NaN
2       2023-10-02 00:02:00  500101001   28   13   1 2023-10-02 00:00:00   NaN
3       2023-10-02 00:03:00  500101001   28   13   1 2023-10-02 00:00:00   NaN
4       2023-10-02 00:04:00  500101001   28   13   1 2023-10-02 00:00:00   NaN
...                     ...        ...  ...  ...  ..                 ...   ...
9710084 2023-12-10 23:15:00  500119091   18    0   1 2023-12-10 23:00:00   NaN
9710085 2023-12-10 23:16:00  500119091   18    0   1 2023-12-10 23:00:00   NaN
9710086 2023-12-10 23:17:00  500119091   18    0   1 2023-12-10 23:00:00   NaN
9710087 2023-12-10 23:18:00  500119091   18    0   1 2023-12-10 23:00:00   NaN
9710088 2023-12-10 23:19:00  500119091   18    0   1 2023-12-10 23:00:00   NaN

[9710089 rows x 7 columns]


In [19]:
df['rain'].fillna(0,inplace=True)
print(df)
df.describe()

                       time        sno  tot  sbi act            datehour  rain
0       2023-10-02 00:00:00  500101001   28   12   1 2023-10-02 00:00:00   0.0
1       2023-10-02 00:01:00  500101001   28   12   1 2023-10-02 00:00:00   0.0
2       2023-10-02 00:02:00  500101001   28   13   1 2023-10-02 00:00:00   0.0
3       2023-10-02 00:03:00  500101001   28   13   1 2023-10-02 00:00:00   0.0
4       2023-10-02 00:04:00  500101001   28   13   1 2023-10-02 00:00:00   0.0
...                     ...        ...  ...  ...  ..                 ...   ...
9710084 2023-12-10 23:15:00  500119091   18    0   1 2023-12-10 23:00:00   0.0
9710085 2023-12-10 23:16:00  500119091   18    0   1 2023-12-10 23:00:00   0.0
9710086 2023-12-10 23:17:00  500119091   18    0   1 2023-12-10 23:00:00   0.0
9710087 2023-12-10 23:18:00  500119091   18    0   1 2023-12-10 23:00:00   0.0
9710088 2023-12-10 23:19:00  500119091   18    0   1 2023-12-10 23:00:00   0.0

[9710089 rows x 7 columns]


,time,tot,sbi,datehour,rain
count,9710089,9.710089e+06,9.710089e+06,9710089,9.710089e+06
mean,2023-11-07 15:09:54.800780800,2.579011e+01,7.598337e+00,2023-11-07 14:40:26.217227264,1.582507e-01
min,2023-10-02 00:00:00,5.000000e+00,0.000000e+00,2023-10-02 00:00:00,0.000000e+00
25%,2023-10-25 03:51:00,1.500000e+01,1.000000e+00,2023-10-25 03:00:00,0.000000e+00
50%,2023-11-09 05:11:00,2.000000e+01,4.000000e+00,2023-11-09 05:00:00,0.000000e+00
75%,2023-11-24 07:24:00,3.000000e+01,1.100000e+01,2023-11-24 07:00:00,0.000000e+00
max,2023-12-10 23:19:00,9.900000e+01,9.900000e+01,2023-12-10 23:00:00,9.000000e+00
std,NaN,1.701816e+01,1.035669e+01,NaN,6.654206e-01


In [87]:
morning_filter = (df["datehour"].dt.hour >= 7) & (df['datehour'].dt.hour <= 21)
rain_hours = df['datehour'][ (df["sno"] == "500101001") & morning_filter &(df['rain'] >= 0.5) ].drop_duplicates() # 0.3 for dribble , 0.5 for small rain
rain_dates = rain_hours.dt.date.drop_duplicates()

print(rain_dates.describe()) # 0.3: 28, 0.5: 15 total:63-62 days

date_rain_hour_cnt = rain_hours.dt.date.value_counts()
print(date_rain_hour_cnt)

count             15
unique            15
top       2023-10-03
freq               1
Name: datehour, dtype: object
datehour
2023-12-04    15
2023-10-04     9
2023-12-06     8
2023-10-08     4
2023-10-20     4
2023-10-03     4
2023-10-05     4
2023-11-16     4
2023-10-28     4
2023-11-26     3
2023-10-06     2
2023-11-30     2
2023-11-24     1
2023-11-12     1
2023-11-14     1
Name: count, dtype: int64


In [ ]:
# Filter out elements that appear less n times
long_rain_dates = rain_dates[rain_dates.isin(date_rain_hour_cnt.index[date_rain_hour_cnt >= 2])]
print(long_rain_dates) # 12
print(long_rain_dates.describe())
rainy_dates = long_rain_dates.array # 12
print(rainy_dates)

[ datetime.date(2023, 10, 3),  datetime.date(2023, 10, 4),
  datetime.date(2023, 10, 5),  datetime.date(2023, 10, 6),
  datetime.date(2023, 10, 8), datetime.date(2023, 10, 20),
 datetime.date(2023, 10, 28), datetime.date(2023, 11, 16),
 datetime.date(2023, 11, 26), datetime.date(2023, 11, 30),
  datetime.date(2023, 12, 4),  datetime.date(2023, 12, 6)]

In [ ]:
old_tb = pd.pivot_table(df, index="time", columns="sno", values="sbi")
tb = (
    pd.pivot_table(df, index="time", columns="sno", values="sbi")
    .resample("20min")
    .agg("first")
)
# exclude long holidays
tb = tb[~tb.index.to_series().dt.date.isin(long_holiday)]
# [] only provides view,so assigning to it cause warning
train = tb[tb.index.to_series().dt.date.isin(date_range(TRAIN_START, TRAIN_END))].copy()
train.reset_index(names="time", inplace=True)
train["weekday"] = train["time"].dt.weekday
train.set_index(["time", "weekday"], inplace=True)

In [ ]:
test = tb[tb.index.to_series().dt.date.isin(date_range(TEST_START, TEST_END))]
y_test = test.values

In [ ]:
result_df = pd.DataFrame(
    columns=ntu_snos,
    index=pd.MultiIndex.from_product(
        [pd.date_range("00:00", "23:59", freq="20min").time, [0, 5, 6]],
        names=("time", "weekday"),
    ),
    dtype=np.float64,
)

In [ ]:
Ein = 0.0
for sno, tot in zip(ntu_snos, ntu_tots):
    # sd = station data
    sd = train[sno].to_frame()
    sd.rename(columns={sno: "sbi"}, inplace=True)
    sd.reset_index(["time", "weekday"], inplace=True)
    sd["date"] = sd["time"].dt.date
    sd["time"] = sd["time"].dt.time
    # print(sd)
    # exit()
    psd = pd.pivot_table(sd, index=["date", "weekday"], columns="time", values="sbi")
    # sno col have its sbi
    # print(psd)
    for day in [0, 5, 6]:  # 0 to 6
        for t in psd.columns:
            # print(t, sno)
            sbi, err = 0, 0
            # print(
            #    psd.loc[psd.index.get_level_values("weekday").isin(range(5)), t].values
            # )
            if day == 0:  # weekday
                sbi, err = optimal_median(
                    y_true=psd.loc[
                        psd.index.get_level_values("weekday").isin(range(5)), t
                    ].values,
                    tot=tot,
                )
            else:
                sbi, err = optimal_median(
                    y_true=psd.loc[
                        psd.index.get_level_values("weekday") == day, t
                    ].values,
                    tot=tot,
                )
            Ein += err
            # print(f"{t} sbi:{sbi}   err: {err}")
            result_df.at[(t, day), sno] = sbi
            # result_df.at[[t, isholiday], sno] = np.float64(sbi)
            # print(result_df.at[t, sno])

print(result_df)<br>
we have avg by #date, now by #sno and #time

In [ ]:
Ein /= result_df.size
print_time_ranges(TRAIN_START, TRAIN_END, TEST_START, TEST_END)
print(f"Ein = {Ein}")

In [ ]:
def trans(s):
    if s in range(5):
        return 0
    return s

self evaluation

In [ ]:
ftr = list(
    np.stack([test.index.time, test.index.to_series().dt.weekday.apply(trans)]).T
)
y_pred = result_df.loc[ftr].values
local_test_range = pd.date_range(TEST_START, TEST_END, freq="20min")

print(y_test)<br>
print(y_pred)<br>
exit()

In [ ]:
evaluation(y_test, y_pred, ntu_tots, local_test_range)

does the same at public test set (2023/10/21 - 2023/10/24)

In [ ]:
public_test_range = pd.date_range(PUBLIC_START, PUBLIC_END, freq="20min")
# list makes indexer 1D, or it is an 2D indexer
ftr = list(
    np.stack([public_test_range.time, np.vectorize(trans)(public_test_range.weekday)]).T
)
y_public_df = result_df.loc[ftr]
print("y_public_df Before")
print(y_public_df)

TODO public forward patch<br>
Set the initial time

In [ ]:
current_datetime = pd.to_datetime("2023-10-20 23:59")
current_time = current_datetime.time()
cur_data = old_tb[old_tb.index == current_datetime]
print(cur_data)
cur = old_tb[old_tb.index.to_series().dt.time == current_time]

Loop to fetch data for the next 20 minutes

In [ ]:
end_datetime = pd.to_datetime("2023-10-21 04:40")
next_datetime = current_datetime
td = timedelta(minutes=20)
total_td = timedelta(minutes=0)
while next_datetime <= end_datetime:
    # Increment current_time by 20 minutes
    next_datetime += td
    next_time = next_datetime.time()
    total_td += td
    # Filter data from old_tb for the current time
    nxt = old_tb[old_tb.index.to_series().dt.time == next_time]
    diff = nxt - cur.shift(freq=total_td)
    mean_diff = pd.pivot_table(diff.mean().reset_index(), columns="sno")
    mean_diff.set_index(cur_data.index, inplace=True)
    upd = cur_data + mean_diff
    # print(upd)
    patch_datetime = next_datetime + timedelta(minutes=1)
    patch_time = patch_datetime.time()
    upd.set_index([[patch_time], [5]], inplace=True)
    y_public_df.loc[(patch_time, 5)] = upd
# TODO public backward patch
current_datetime = pd.to_datetime("2023-10-25 00:00")
current_time = current_datetime.time()
cur_data = old_tb[old_tb.index == current_datetime]
print(cur_data)
cur = old_tb[old_tb.index.to_series().dt.time == current_time]

Loop to fetch data for the next 20 minutes

In [ ]:
end_datetime = pd.to_datetime("2023-10-24 22:00")
prev_datetime = current_datetime
td = timedelta(minutes=20)
total_td = timedelta(minutes=0)
while prev_datetime >= end_datetime:
    # Increment current_time by 20 minutes
    prev_datetime -= td
    prev_time = prev_datetime.time()
    total_td -= td
    # Filter data from old_tb for the current time
    nxt = old_tb[old_tb.index.to_series().dt.time == prev_time]
    diff = nxt - cur.shift(freq=total_td)
    mean_diff = pd.pivot_table(diff.mean().reset_index(), columns="sno")
    mean_diff.set_index(cur_data.index, inplace=True)  # for add to cur_data only
    upd = cur_data + mean_diff
    # print(upd)
    patch_datetime = prev_datetime  #  + timedelta(minutes=1) # only needed when 23:59
    patch_time = patch_datetime.time()
    # second index is weekday 10/24 Tue
    upd.set_index([[patch_time], [0]], inplace=True)
    y_public_df.loc[(patch_time, 0)] = upd

ake value in range

In [ ]:
for col, tot in zip(y_public_df.columns, ntu_tots):
    y_public_df[col] = y_public_df[col].clip(lower=0, upper=tot)

In [ ]:
print("y_public_df After")
print(y_public_df)

In [ ]:
y_public_test = y_public_df.values
public_test_df = pd.DataFrame(y_public_test, columns=ntu_snos, index=public_test_range)

In [ ]:
private_test_range = pd.date_range(PRIVATE_START, PRIVARE_END, freq="20min")
ftr = list(
    np.stack(
        [private_test_range.time, np.vectorize(trans)(private_test_range.weekday)]
    ).T
)
y_private_df = result_df.loc[ftr]

TODO patch private<br>
Set the initial time


<br>
current_datetime = pd.to_datetime("2023-12-10 23:40")<br>
current_time = current_datetime.time()<br>
cur_data = old_tb[old_tb.index == current_datetime]<br>
print(cur_data)<br>
cur = old_tb[old_tb.index.to_series().dt.time == current_time]<br>
# Loop to fetch data for the next 20 minutes<br>
end_datetime = pd.to_datetime("2023-12-04 03:59")<br>
next_datetime = current_datetime<br>
td = timedelta(minutes=20)<br>
total_td = timedelta(minutes=0)<br>
while next_datetime <= end_datetime:<br>
    # Increment current_time by 20 minutes<br>
    next_datetime += td<br>
    next_time = next_datetime.time()<br>
    total_td += td<br>
    # Filter data from old_tb for the current time<br>
    nxt = old_tb[old_tb.index.to_series().dt.time == next_time]<br>
    diff = nxt - cur.shift(freq=total_td)<br>
    mean_diff = pd.pivot_table(diff.mean().reset_index(), columns="sno")<br>
    mean_diff.set_index(cur_data.index, inplace=True)<br>
    upd = cur_data + mean_diff<br>
    # print(upd)<br>
    patch_datetime = next_datetime + timedelta(minutes=1)<br>
    patch_time = patch_datetime.time()<br>
    upd.set_index([[patch_time], [0]], inplace=True)<br>
    y_private_df.loc[(patch_time, 0)] = upd<br>
print(y_private_df)<br>


In [ ]:
y_private_test = y_private_df.values
private_test_df = pd.DataFrame(
    y_private_test, columns=ntu_snos, index=private_test_range
)

convert the prediction to the required format

In [ ]:
tmp = pd.concat(
    [
        public_test_df,
        private_test_df,
    ]
)
# reset_index: old index => "time" column
# id_vars: fixed column like index
# var_name: columns.name to "sno" column
# value_name: value => "sbi" column
tmp = tmp.reset_index(names="time").melt(
    id_vars="time", var_name="sno", value_name="sbi"
)
out_df = pd.DataFrame(
    {
        "id": (
            tmp["time"].dt.strftime("%Y%m%d")
            + "_"
            + tmp["sno"]
            + "_"
            + tmp["time"].dt.strftime("%H:%M")
        ),
        "sbi": tmp["sbi"],
    }
)
out_df.to_csv(
    f"../submission/pub_pri_{datetime.now().strftime('%m-%d-%H-%M')}.csv", index=False
)
print("csv created")